<a href="https://colab.research.google.com/github/whyonce/testgit/blob/main/Six.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!head /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 85
model name	: Intel(R) Xeon(R) CPU @ 2.00GHz
stepping	: 3
microcode	: 0x1
cpu MHz		: 2000.134
cache size	: 39424 KB
physical id	: 0


In [ ]:
pwd

'/content'

In [ ]:
import os, re 
import numpy as np
import tensorflow as tf

# 파일을 읽기모드로 열고
# 라인 단위로 끊어서 list 형태로 읽어옵니다.
# 구글 코랩 기준
file_path = '/content/drive/MyDrive/Colab Notebooks/lyricist/data/shakespeare.txt'
# 깃헙 제출용 기준
# file_path = 'lyricist/data/shakespeare.txt'
with open(file_path, "r") as f:
    raw_corpus = f.read().splitlines()

# 앞에서부터 10라인만 화면에 출력해 볼까요?
print(raw_corpus[:9])

['First Citizen:', 'Before we proceed any further, hear me speak.', '', 'All:', 'Speak, speak.', '', 'First Citizen:', 'You are all resolved rather to die than to famish?', '']


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.

    if idx > 9: break   # 일단 문장 10개만 확인해 볼 겁니다.
        
    print(sentence)

Before we proceed any further, hear me speak.
Speak, speak.
You are all resolved rather to die than to famish?


In [ ]:
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
# 이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있겠네요!
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample        sentence.  "))


<start> this is sample sentence . <end>


In [ ]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]

['<start> before we proceed any further , hear me speak . <end>',
 '<start> speak , speak . <end>',
 '<start> you are all resolved rather to die than to famish ? <end>',
 '<start> resolved . resolved . <end>',
 '<start> first , you know caius marcius is chief enemy to the people . <end>',
 '<start> we know t , we know t . <end>',
 '<start> let us kill him , and we ll have corn at our own price . <end>',
 '<start> is t a verdict ? <end>',
 '<start> no more talking on t let it be done away , away ! <end>',
 '<start> one word , good citizens . <end>']

In [ ]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용합니다
# 더 잘 알기 위해 아래 문서들을 참고하면 좋습니다
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences
def tokenize(corpus):
    # 7000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 7000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=7000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2  143   40 ...    0    0    0]
 [   2  110    4 ...    0    0    0]
 [   2   11   50 ...    0    0    0]
 ...
 [   2  149 4553 ...    0    0    0]
 [   2   34   71 ...    0    0    0]
 [   2  945   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f61588aff50>


In [ ]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[  2 143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0
   0   0]
[143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0   0
   0   0]


In [ ]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset element_spec=(TensorSpec(shape=(256, 20), dtype=tf.int32, name=None), TensorSpec(shape=(256, 20), dtype=tf.int32, name=None))>

In [ ]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [ ]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
# 지금은 동작 원리에 너무 빠져들지 마세요~
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 20, 7001), dtype=float32, numpy=
array([[[ 5.92987308e-05,  3.35658347e-04, -2.52945010e-05, ...,
         -1.07262109e-04, -1.69968960e-04,  2.04169264e-04],
        [ 2.14046435e-04,  7.26153376e-04, -1.24206999e-04, ...,
          9.32197054e-06, -5.73392663e-06,  5.90032083e-04],
        [ 2.50723213e-04,  1.10364787e-03, -1.37655312e-04, ...,
          4.17442643e-04,  2.78050487e-04,  9.10501054e-04],
        ...,
        [ 2.22843955e-03, -1.69194260e-04,  3.01103810e-05, ...,
          8.46385083e-04,  7.85199401e-04,  1.14758448e-04],
        [ 2.81504076e-03, -6.23065862e-05,  2.18120113e-04, ...,
          1.08408241e-03,  8.87150818e-04, -8.49695352e-05],
        [ 3.36338859e-03,  9.05921552e-05,  3.52052419e-04, ...,
          1.29221100e-03,  9.84928571e-04, -2.91521050e-04]],

       [[ 5.92987308e-05,  3.35658347e-04, -2.52945010e-05, ...,
         -1.07262109e-04, -1.69968960e-04,  2.04169264e-04],
        [-4.46643526e-06,  5.73799538e-04,  7.

In [ ]:
model.summary()

Model: "text_generator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  1792256   
                                                                 
 lstm (LSTM)                 multiple                  5246976   
                                                                 
 lstm_1 (LSTM)               multiple                  8392704   
                                                                 
 dense (Dense)               multiple                  7176025   
                                                                 
Total params: 22,607,961
Trainable params: 22,607,961
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# optimizer와 loss등은 차차 배웁니다
# 혹시 미리 알고 싶다면 아래 문서를 참고하세요
# https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
# https://www.tensorflow.org/api_docs/python/tf/keras/losses
# 양이 상당히 많은 편이니 지금 보는 것은 추천하지 않습니다
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)

Epoch 1/10
93/93 [==============================] - 19s 168ms/step - loss: 3.4916
Epoch 2/10
93/93 [==============================] - 16s 171ms/step - loss: 2.8091
Epoch 3/10
93/93 [==============================] - 16s 172ms/step - loss: 2.7115
Epoch 4/10
93/93 [==============================] - 16s 174ms/step - loss: 2.6132
Epoch 5/10
93/93 [==============================] - 16s 176ms/step - loss: 2.5498
Epoch 6/10
93/93 [==============================] - 17s 177ms/step - loss: 2.5021
Epoch 7/10
93/93 [==============================] - 17s 179ms/step - loss: 2.4553
Epoch 8/10
93/93 [==============================] - 17s 182ms/step - loss: 2.4068
Epoch 9/10
93/93 [==============================] - 17s 183ms/step - loss: 2.3599
Epoch 10/10
93/93 [==============================] - 17s 183ms/step - loss: 2.3170


In [ ]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [ ]:
generate_text(model, tokenizer, init_sentence="<start> he")

'<start> he is a <unk> , and i have not been <end> '

# 이제 lyricist 파일의 자료를 사용해 모델을 학습시켜 보겠습니다.

In [1]:
import os, re 
import numpy as np
import tensorflow as tf
import glob

# 구글 코랩 기준
txt_file_path =  '/content/drive/MyDrive/Colab Notebooks/lyricist/data/lyrics/*'
# 깃헙 제출용 기준
# txt_file_path =  'lyricist/data/lyrics/*.txt'
txt_list = glob.glob(txt_file_path)

raw_corpus1 = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    # 구글 코랩은 인코딩 오류 없이 되어서 밑에 있는 코드로 되었다.
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus1.extend(raw)
    # 만약 주피터 실행시 인코딩 오류가 뜨면 밑에 있는 코드를 쓰도록 하자
    #f = open(txt_file, 'r', encoding="UTF-8")
    #raw = f.read().splitlines()
    #raw_corpus1.extend(raw)

print("데이터 크기:", len(raw_corpus1))
print("Examples:\n", raw_corpus1[:3])

데이터 크기: 187088
Examples:
 ['Looking for some education', 'Made my way into the night', 'All that bullshit conversation']


# 데이터 정제

In [3]:
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
# 이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있겠네요!
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample        sentence.  "))


<start> this is sample sentence . <end>


In [4]:
# 여기에 정제된 문장을 모을겁니다
corpus1 = []

for sentence in raw_corpus1:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus1.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus1[:10]

['<start> looking for some education <end>',
 '<start> made my way into the night <end>',
 '<start> all that bullshit conversation <end>',
 '<start> baby , can t you read the signs ? i won t bore you with the details , baby <end>',
 '<start> i don t even wanna waste your time <end>',
 '<start> let s just say that maybe <end>',
 '<start> you could help me ease my mind <end>',
 '<start> i ain t mr . right but if you re looking for fast love <end>',
 '<start> if that s love in your eyes <end>',
 '<start> it s more than enough <end>']

In [ ]:
temp = corpus1.copy()

In [ ]:
temp = np.array(temp)

In [ ]:
temp.shape

(175749,)

In [ ]:
len(corpus1)

175749

In [ ]:
len(corpus1[0])

40

In [ ]:
temp_len = np.array([])
for i in range(len(corpus1)):
  temp_len = np.append(temp_len, len(corpus1[i]))
print(temp_len[:10])

[40. 40. 44. 88. 48. 39. 44. 68. 41. 35.]


In [ ]:
import pandas as pd
data_df = pd.DataFrame(temp_len)

In [ ]:
from IPython.display import display, Image
import matplotlib.pyplot as plt
print(data_df.mean())
print(data_df.max())
print(data_df.min())
print(data_df.std())


0    51.26073
dtype: float64
0    1478.0
dtype: float64
0    14.0
dtype: float64
0    20.555508
dtype: float64


In [ ]:
Q3, Q1 = np.percentile(data_df, [75 ,25])
IQR = Q3 - Q1
IQR

23.0

In [5]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용합니다
# 더 잘 알기 위해 아래 문서들을 참고하면 좋습니다
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences
def tokenize(corpus):
    # 7000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 7000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=7000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus1)

[[  2 304  28 ...   0   0   0]
 [  2 221  13 ...   0   0   0]
 [  2  24  17 ...   0   0   0]
 ...
 [  2   5  33 ...   0   0   0]
 [  2   5  33 ...   0   0   0]
 [  2   5 371 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7f97d735d550>


In [ ]:
tensor.shape

(175749, 347)

In [ ]:
print(tokenizer.index_word)

{1: '<unk>', 2: '<start>', 3: '<end>', 4: ',', 5: 'i', 6: 'the', 7: 'you', 8: 'and', 9: 'a', 10: 'to', 11: 'it', 12: 'me', 13: 'my', 14: 'in', 15: 't', 16: 's', 17: 'that', 18: 'on', 19: 'of', 20: '.', 21: 'your', 22: 'm', 23: 'we', 24: 'all', 25: 'like', 26: 'is', 27: 'be', 28: 'for', 29: 'up', 30: 'so', 31: 'with', 32: 'can', 33: 'love', 34: 'know', 35: 'just', 36: 'but', 37: 'don', 38: 'no', 39: 'oh', 40: 'what', 41: 'got', 42: 'this', 43: '?', 44: 'get', 45: 'they', 46: 'when', 47: 'do', 48: 'she', 49: 'yeah', 50: 'now', 51: 'baby', 52: 'if', 53: 'go', 54: 're', 55: 'he', 56: 'out', 57: 'was', 58: 'one', 59: '!', 60: 'down', 61: 'll', 62: 'want', 63: 'let', 64: 'see', 65: 'there', 66: 'come', 67: 'cause', 68: 'say', 69: 'her', 70: 'not', 71: 'at', 72: 'make', 73: 'time', 74: 'from', 75: 'back', 76: 'have', 77: 'are', 78: 'how', 79: 'never', 80: 'girl', 81: 'as', 82: 'man', 83: 'take', 84: 'right', 85: 'way', 86: 'wanna', 87: 'im', 88: 'will', 89: 'ain', 90: 'need', 91: 've', 92: 't

In [ ]:
print(len(tokenizer.index_word))

27592


# 일단 그래서 토큰화 후 경우를 3가지로 나눠서 해 보겠습니다.

# 1. 아이펠 기준 2. 15보다 클 경우 3. 15보다 작을 경우


# 1. 아이펠 기준

In [ ]:
tensor[0].shape

(347,)

In [ ]:
tgt_input1.shape

(175749, 15)

In [ ]:
temp = tensor.copy()
temp = temp[:,:15]

In [ ]:
temp.shape

(175749, 15)

In [ ]:
temp[2][14]

0

In [ ]:
tensor1 = np.array([])
for i in range(x):
  if temp[i][14] == 0 or temp[i][14] == 3:
    tensor1 = np.append(tensor1, temp[i])
print(tensor1.shape)

(2340195,)


In [ ]:
tensor1 = np.reshape(tensor1, (156013,15))
tensor1.shape

(156013, 15)

In [ ]:
src_input1 = tensor1[:,:-1] 
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input1 = tensor1[:, 1:]  

In [ ]:
print(src_input1.shape)
print(tgt_input1.shape)

(156013, 14)
(156013, 14)


In [ ]:
from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input1, tgt_input1, test_size=0.3, random_state=80)

In [ ]:
enc_train1, enc_val1, dec_train1, dec_val1 = train_test_split(src_input1, tgt_input1, test_size=0.2, random_state=80)

In [ ]:
enc_train1.shape

(124810, 14)

In [ ]:
enc_val.shape

(46804, 14)

In [ ]:
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 256
steps_per_epoch = len(enc_train) // BATCH_SIZE

BUFFER_SIZE_v = len(enc_val)
BATCH_SIZE_v = 256
steps_per_epoch_v = len(enc_val) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset

# 테스트 데이터 30퍼
dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

dataset_v = tf.data.Dataset.from_tensor_slices((enc_val, dec_val))
dataset_v = dataset_v.shuffle(BUFFER_SIZE_v)
dataset_v = dataset_v.batch(BATCH_SIZE_v, drop_remainder=True)

#테스트 데이터 20퍼
dataset1 = tf.data.Dataset.from_tensor_slices((enc_train1, dec_train1))
dataset1 = dataset1.shuffle(BUFFER_SIZE)
dataset1 = dataset1.batch(BATCH_SIZE, drop_remainder=True)

dataset1_v = tf.data.Dataset.from_tensor_slices((enc_val1, dec_val1))
dataset1_v = dataset1_v.shuffle(BUFFER_SIZE_v)
dataset1_v = dataset1_v.batch(BATCH_SIZE_v, drop_remainder=True)

In [ ]:
print((dataset))
print((dataset_v))

<BatchDataset element_spec=(TensorSpec(shape=(256, 14), dtype=tf.float64, name=None), TensorSpec(shape=(256, 14), dtype=tf.float64, name=None))>
<BatchDataset element_spec=(TensorSpec(shape=(256, 14), dtype=tf.float64, name=None), TensorSpec(shape=(256, 14), dtype=tf.float64, name=None))>


In [ ]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

# 테스트 데이터 30퍼

In [ ]:
# optimizer와 loss등은 차차 배웁니다
# 혹시 미리 알고 싶다면 아래 문서를 참고하세요
# https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
# https://www.tensorflow.org/api_docs/python/tf/keras/losses
# 양이 상당히 많은 편이니 지금 보는 것은 추천하지 않습니다
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10, validation_data = dataset_v)

Epoch 1/10
426/426 [==============================] - 45s 86ms/step - loss: 3.3716 - val_loss: 3.0039
Epoch 2/10
426/426 [==============================] - 36s 84ms/step - loss: 2.8871 - val_loss: 2.7972
Epoch 3/10
426/426 [==============================] - 36s 84ms/step - loss: 2.6981 - val_loss: 2.6694
Epoch 4/10
426/426 [==============================] - 36s 84ms/step - loss: 2.5493 - val_loss: 2.5793
Epoch 5/10
426/426 [==============================] - 36s 84ms/step - loss: 2.4158 - val_loss: 2.5065
Epoch 6/10
426/426 [==============================] - 36s 84ms/step - loss: 2.2885 - val_loss: 2.4489
Epoch 7/10
426/426 [==============================] - 36s 84ms/step - loss: 2.1663 - val_loss: 2.3994
Epoch 8/10
426/426 [==============================] - 36s 84ms/step - loss: 2.0483 - val_loss: 2.3584
Epoch 9/10
426/426 [==============================] - 36s 84ms/step - loss: 1.9313 - val_loss: 2.3240
Epoch 10/10
426/426 [==============================] - 36s 84ms/step - loss: 1.817

# 테스트 데이터 20퍼


In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset1, epochs=10, validation_data = dataset1_v)

Epoch 1/10
487/487 [==============================] - 41s 81ms/step - loss: 1.7999 - val_loss: 2.2724
Epoch 2/10
487/487 [==============================] - 39s 79ms/step - loss: 1.6588 - val_loss: 2.2494
Epoch 3/10
487/487 [==============================] - 39s 79ms/step - loss: 1.5488 - val_loss: 2.2314
Epoch 4/10
487/487 [==============================] - 39s 79ms/step - loss: 1.4474 - val_loss: 2.2209
Epoch 5/10
487/487 [==============================] - 39s 79ms/step - loss: 1.3560 - val_loss: 2.2200
Epoch 6/10
487/487 [==============================] - 39s 79ms/step - loss: 1.2757 - val_loss: 2.2231
Epoch 7/10
487/487 [==============================] - 39s 79ms/step - loss: 1.2064 - val_loss: 2.2302
Epoch 8/10
487/487 [==============================] - 39s 80ms/step - loss: 1.1480 - val_loss: 2.2395
Epoch 9/10
487/487 [==============================] - 39s 80ms/step - loss: 1.1004 - val_loss: 2.2565
Epoch 10/10
487/487 [==============================] - 39s 79ms/step - loss: 1.061

# loss값이 더 줄은 걸 확인 할 수 있었다.

In [ ]:
tensor

array([[  2, 304,  28, ...,   0,   0,   0],
       [  2, 221,  13, ...,   0,   0,   0],
       [  2,  24,  17, ...,   0,   0,   0],
       ...,
       [  2,   5,  33, ...,   0,   0,   0],
       [  2,   5,  33, ...,   0,   0,   0],
       [  2,   5, 371, ...,   0,   0,   0]], dtype=int32)

In [ ]:
temp = tensor.copy()

In [ ]:
x, y = temp.shape

In [ ]:
x

175749

In [ ]:
print(tokenizer.index_word[1])
print(tokenizer.index_word[2])
print(tokenizer.index_word[3])

<unk>
<start>
<end>


In [ ]:
a = np.where(temp[0] == 3)

In [ ]:
type(a)

int

In [ ]:
a = int(a[0])

In [ ]:
temp[0][5]

3

# 위 코드로 문장의 끝부분을 발견할 수 있었다.
##### 그래서 이 코드를 조건문으로 사용해 end인 위치값을 구해 start,end를 포함해 몇개의 단어로 이루져 있는지 셀 예정이다.

In [ ]:
count = np.array([])
for i in range(x):
  count_temp = np.where(temp[i] == 3) 
  count_temp = int(count_temp[0])
  count = np.append(count, count_temp) 

print(count)


[ 5.  7.  5. ...  5. 14.  8.]


In [ ]:
count[8000]

9.0

In [ ]:
import pandas as pd
data_df = pd.DataFrame(count)

print(data_df.mean())
print(data_df.max())
print(data_df.min())
print(data_df.std())

Q3, Q1 = np.percentile(data_df, [75 ,25])
IQR = Q3 - Q1
IQR

0    9.520703
dtype: float64
0    346.0
dtype: float64
0    1.0
dtype: float64
0    4.809925
dtype: float64


6.0

# 여기서 문장 단어 갯수의 평균을 보면 9.5가 나온다.

# 15기준으로 했을 경우 175749 -> 156013 으로 약 2만개가 줄었다.

## 그래서 아이펠 기준으로 데이터를 늘리는 것이 나은지 아니면 줄이는게 맞는지 볼려고 한다.

In [ ]:
temp = tensor.copy()
temp1 = tensor.copy()
temp2 = tensor.copy()
temp3 = tensor.copy()
temp4 = tensor.copy()
temp = temp[:,:16]
temp1 = temp1[:,:17]
temp2 = temp2[:,:18]
temp3 = temp3[:,:19]
temp4 = temp4[:,:20]
print(temp.shape)

(175749, 16)


In [ ]:
tensor1 = np.array([])
for i in range(x):
  if temp[i][15] == 0 or temp[i][15] == 3:
    tensor1 = np.append(tensor1, temp[i])
print(tensor1.shape)

tensor2 = np.array([])
for i in range(x):
  if temp1[i][16] == 0 or temp1[i][16] == 3:
    tensor2 = np.append(tensor2, temp1[i])
print(tensor2.shape)

tensor3 = np.array([])
for i in range(x):
  if temp2[i][17] == 0 or temp2[i][17] == 3:
    tensor3 = np.append(tensor3, temp2[i])
print(tensor3.shape)

tensor4 = np.array([])
for i in range(x):
  if temp3[i][18] == 0 or temp3[i][18] == 3:
    tensor4 = np.append(tensor4, temp3[i])
print(tensor4.shape)

tensor5 = np.array([])
for i in range(x):
  if temp4[i][19] == 0 or temp4[i][19] == 3:
    tensor5 = np.append(tensor5, temp4[i])
print(tensor5.shape)

(2564112,)
(2779296,)
(2987892,)
(3192798,)
(3391060,)


In [ ]:
tensor1 = np.reshape(tensor1, (160257,16))
print(tensor1.shape)

tensor2 = np.reshape(tensor2, (163488,17))
print(tensor2.shape)

tensor3 = np.reshape(tensor3, (165994,18))
print(tensor3.shape)

tensor4 = np.reshape(tensor4, (168042,19))
print(tensor4.shape)

tensor5 = np.reshape(tensor5, (169553,20))
print(tensor5.shape)


(160257, 16)
(163488, 17)
(165994, 18)
(168042, 19)
(169553, 20)


In [ ]:
src_input1 = tensor1[:,:-1] 
tgt_input1 = tensor1[:, 1:]

src_input2 = tensor2[:,:-1] 
tgt_input2 = tensor2[:, 1:]  

src_input3 = tensor3[:,:-1] 
tgt_input3 = tensor3[:, 1:]  

src_input4 = tensor4[:,:-1] 
tgt_input4 = tensor4[:, 1:]  

src_input5 = tensor5[:,:-1] 
tgt_input5 = tensor5[:, 1:]  

In [ ]:
enc_train1, enc_val1, dec_train1, dec_val1 = train_test_split(src_input1, tgt_input1, test_size=0.2, random_state=80)

enc_train2, enc_val2, dec_train2, dec_val2 = train_test_split(src_input2, tgt_input2, test_size=0.2, random_state=80)

enc_train3, enc_val3, dec_train3, dec_val3 = train_test_split(src_input3, tgt_input3, test_size=0.2, random_state=80)

enc_train4, enc_val4, dec_train4, dec_val4 = train_test_split(src_input4, tgt_input4, test_size=0.2, random_state=80)

enc_train5, enc_val5, dec_train5, dec_val5 = train_test_split(src_input5, tgt_input5, test_size=0.2, random_state=80)


In [ ]:
def buf(enc_train, enc_val, dec_train, dec_val, words):
  BUFFER_SIZE = len(enc_train)
  BATCH_SIZE = 256
  steps_per_epoch = len(enc_train) // BATCH_SIZE

  BUFFER_SIZE_v = len(enc_val)
  BATCH_SIZE_v = 256
  steps_per_epoch_v = len(enc_val) // BATCH_SIZE



  # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
  VoCAB_SIZE = words
  #VOCAB_SIZE = tokenizer.num_words + 1   

  dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
  dataset = dataset.shuffle(BUFFER_SIZE)
  dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

  dataset_v = tf.data.Dataset.from_tensor_slices((enc_val, dec_val))
  dataset_v = dataset_v.shuffle(BUFFER_SIZE_v)
  dataset_v = dataset_v.batch(BATCH_SIZE_v, drop_remainder=True)

  return dataset, dataset_v

In [ ]:
dataset1, dataset_v1 = buf(enc_train1, enc_val1, dec_train1, dec_val1, tokenizer.num_words)
dataset2, dataset_v2 = buf(enc_train2, enc_val2, dec_train2, dec_val2, tokenizer.num_words) 
dataset3, dataset_v3 = buf(enc_train3, enc_val3, dec_train3, dec_val3, tokenizer.num_words)
dataset4, dataset_v4 = buf(enc_train4, enc_val4, dec_train4, dec_val4, tokenizer.num_words)
dataset5, dataset_v5 = buf(enc_train5, enc_val5, dec_train5, dec_val5, tokenizer.num_words)

In [ ]:
embedding_size = 256
hidden_size = 1024
model1 = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)
model2 = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)
model3 = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)
model4 = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)
model5 = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate= 0.001)
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

In [ ]:
model1.compile(loss=loss, optimizer=optimizer)
model1.fit(dataset1, epochs=10, validation_data = dataset_v1)

model2.compile(loss=loss, optimizer=optimizer)
model2.fit(dataset2, epochs=10, validation_data = dataset_v)

model3.compile(loss=loss, optimizer=optimizer)
model3.fit(dataset3, epochs=10, validation_data = dataset_v3)

model4.compile(loss=loss, optimizer=optimizer)
model4.fit(dataset4, epochs=10, validation_data = dataset_v4)

model5.compile(loss=loss, optimizer=optimizer)
model5.fit(dataset5, epochs=10, validation_data = dataset_v5)

Epoch 1/10
500/500 [==============================] - 48s 88ms/step - loss: 3.1956 - val_loss: 2.8908
Epoch 2/10
500/500 [==============================] - 44s 87ms/step - loss: 2.7664 - val_loss: 2.6974
Epoch 3/10
500/500 [==============================] - 44s 87ms/step - loss: 2.6030 - val_loss: 2.5900
Epoch 4/10
500/500 [==============================] - 43s 87ms/step - loss: 2.4842 - val_loss: 2.5117
Epoch 5/10
500/500 [==============================] - 43s 87ms/step - loss: 2.3829 - val_loss: 2.4492
Epoch 6/10
500/500 [==============================] - 43s 87ms/step - loss: 2.2924 - val_loss: 2.4000
Epoch 7/10
500/500 [==============================] - 44s 87ms/step - loss: 2.2069 - val_loss: 2.3574
Epoch 8/10
500/500 [==============================] - 43s 87ms/step - loss: 2.1253 - val_loss: 2.3202
Epoch 9/10
500/500 [==============================] - 44s 87ms/step - loss: 2.0473 - val_loss: 2.2902
Epoch 10/10
500/500 [==============================] - 43s 87ms/step - loss: 1.971

In [ ]:
print(generate_text(model1, tokenizer, init_sentence="<start> he"))

print(generate_text(model2, tokenizer, init_sentence="<start> he"))

print(generate_text(model3, tokenizer, init_sentence="<start> he"))

print(generate_text(model4, tokenizer, init_sentence="<start> he"))

print(generate_text(model5, tokenizer, init_sentence="<start> he"))


<start> he s a <unk> , and a <unk> <end> 
<start> he s a <unk> in the dark <end> 
<start> he s a <unk> in the <unk> <end> 
<start> he s lookin for the fuse <end> 
<start> he s a <unk> <end> 


In [ ]:
print(generate_text(model1, tokenizer, init_sentence="<start> some"))

print(generate_text(model2, tokenizer, init_sentence="<start> some"))

print(generate_text(model3, tokenizer, init_sentence="<start> some"))

print(generate_text(model4, tokenizer, init_sentence="<start> some"))

print(generate_text(model5, tokenizer, init_sentence="<start> some"))


<start> some people want diamond rings <end> 
<start> some people live just for the fame <end> 
<start> some people want diamond rings <end> 
<start> some people want diamond rings <end> 
<start> some people want diamond rings <end> 


In [ ]:
print(generate_text(model1, tokenizer, init_sentence="<start> i love"))

print(generate_text(model2, tokenizer, init_sentence="<start> i love"))

print(generate_text(model3, tokenizer, init_sentence="<start> i love"))

print(generate_text(model4, tokenizer, init_sentence="<start> i love"))

print(generate_text(model5, tokenizer, init_sentence="<start> i love"))


<start> i love you <end> 
<start> i love you , i love you <end> 
<start> i love you , i love you , i love you , i m <end> 
<start> i love you so , so , i do <end> 
<start> i love you , i love you so much <end> 


In [ ]:
print(generate_text(model1, tokenizer, init_sentence="<start> abc"))

print(generate_text(model2, tokenizer, init_sentence="<start> aiffel"))

print(generate_text(model3, tokenizer, init_sentence="<start> green"))

print(generate_text(model4, tokenizer, init_sentence="<start> grass"))

print(generate_text(model5, tokenizer, init_sentence="<start> star"))


<start> <unk> <unk> <unk> <unk> <unk> <end> 
<start> <unk> <unk> <end> 
<start> green eggs and ham <end> 
<start> grass is the only one who knows <end> 
<start> star to be the same <end> 


In [ ]:
print(generate_text(model5, tokenizer, init_sentence="<start> tak"))


<start> take me home , <end> 


### loss값은 낮았지만 필자가 원하는 문장이 없기 때문에 이번에는 단어 수를 무진장 늘리고 해보기로 했다.

In [ ]:
temp = tensor.copy()
temp1 = tensor.copy()
temp2 = tensor.copy()
temp3 = tensor.copy()
temp4 = tensor.copy()
temp = temp[:,:30]
temp1 = temp1[:,:40]
temp2 = temp2[:,:50]
temp3 = temp3[:,:60]
temp4 = temp4[:,:70]
print(temp.shape)

(175749, 30)


In [7]:
temp = tensor.copy()
x,y = temp.shape
t = np.empty((0,1), int)
for i in range(x):
    t = np.append(t, temp[i][29])
print(t.shape)



(175749,)


In [9]:
temp = temp[:,:30]

In [10]:

tensor1 = np.empty((0,30), int)
for i in range(x):
    if t[i] == 0 or t[i] == 3:
        temp4 = np.array([temp[i]])
        tensor1 = np.append(tensor1, temp4, axis = 0)
print(tensor1.shape)

(175175, 30)


# 위 코드는 필자가 처음에는 아래 코드로 데이터 처리를 했는데

# 반복문 하나당 평균 40분이 넘게 걸려 총 2시간이 넘게 걸렸다.

# 그래서 어떻게 하면 실행시간을 줄일 수 있을까 여러 테스트를 하던 중 
# 위 코드로 실험을 해보니 15분으로 실행시간을 반절 이상 줄였다.


In [ ]:
tensor1 = np.array([])
for i in range(x):
  if temp[i][29] == 0 or temp[i][29] == 3:
    tensor1 = np.append(tensor1, temp[i])
print(tensor1.shape)

tensor2 = np.array([])
for i in range(x):
  if temp1[i][39] == 0 or temp1[i][39] == 3:
    tensor2 = np.append(tensor2, temp1[i])
print(tensor2.shape)

tensor3 = np.array([])
for i in range(x):
  if temp2[i][49] == 0 or temp2[i][49] == 3:
    tensor3 = np.append(tensor3, temp2[i])
print(tensor3.shape)

tensor4 = np.array([])
for i in range(x):
  if temp3[i][59] == 0 or temp3[i][59] == 3:
    tensor4 = np.append(tensor4, temp3[i])
print(tensor4.shape)

tensor5 = np.array([])
for i in range(x):
  if temp4[i][69] == 0 or temp4[i][69] == 3:
    tensor5 = np.append(tensor5, temp4[i])
print(tensor5.shape)

(5255250,)
(7024400,)
(8783650,)
(10542600,)
(12300820,)


In [ ]:
tensor1 = np.reshape(tensor1, (175175,30))
print(tensor1.shape)

tensor2 = np.reshape(tensor2, (175610,40))
print(tensor2.shape)

tensor3 = np.reshape(tensor3, (175673,50))
print(tensor3.shape)

tensor4 = np.reshape(tensor4, (175710,60))
print(tensor4.shape)

tensor5 = np.reshape(tensor5, (175726,70))
print(tensor5.shape)

(175175, 30)
(175610, 40)
(175673, 50)
(175710, 60)
(175726, 70)


In [ ]:
src_input1 = tensor1[:,:-1] 
tgt_input1 = tensor1[:, 1:]

src_input2 = tensor2[:,:-1] 
tgt_input2 = tensor2[:, 1:]  

src_input3 = tensor3[:,:-1] 
tgt_input3 = tensor3[:, 1:]  

src_input4 = tensor4[:,:-1] 
tgt_input4 = tensor4[:, 1:]  

src_input5 = tensor5[:,:-1] 
tgt_input5 = tensor5[:, 1:]  

dataset1, dataset_v1 = buf(enc_train1, enc_val1, dec_train1, dec_val1, tokenizer.num_words)
dataset2, dataset_v2 = buf(enc_train2, enc_val2, dec_train2, dec_val2, tokenizer.num_words) 
dataset3, dataset_v3 = buf(enc_train3, enc_val3, dec_train3, dec_val3, tokenizer.num_words)
dataset4, dataset_v4 = buf(enc_train4, enc_val4, dec_train4, dec_val4, tokenizer.num_words)
dataset5, dataset_v5 = buf(enc_train5, enc_val5, dec_train5, dec_val5, tokenizer.num_words)

In [ ]:
embedding_size = 256
hidden_size = 1024
model1 = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)
model2 = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)
model3 = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)
model4 = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)
model5 = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

optimizer = tf.keras.optimizers.Adam(learning_rate= 0.001)
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model1.compile(loss=loss, optimizer=optimizer)
model1.fit(dataset1, epochs=10, validation_data = dataset_v1)

model2.compile(loss=loss, optimizer=optimizer)
model2.fit(dataset2, epochs=10, validation_data = dataset_v)

model3.compile(loss=loss, optimizer=optimizer)
model3.fit(dataset3, epochs=10, validation_data = dataset_v3)

model4.compile(loss=loss, optimizer=optimizer)
model4.fit(dataset4, epochs=10, validation_data = dataset_v4)

model5.compile(loss=loss, optimizer=optimizer)
model5.fit(dataset5, epochs=10, validation_data = dataset_v5)

Epoch 1/10
500/500 [==============================] - 48s 88ms/step - loss: 3.2342 - val_loss: 2.9315
Epoch 2/10
500/500 [==============================] - 44s 87ms/step - loss: 2.8188 - val_loss: 2.7496
Epoch 3/10
500/500 [==============================] - 44s 87ms/step - loss: 2.6526 - val_loss: 2.6296
Epoch 4/10
500/500 [==============================] - 44s 87ms/step - loss: 2.5338 - val_loss: 2.5503
Epoch 5/10
500/500 [==============================] - 44s 87ms/step - loss: 2.4362 - val_loss: 2.4863
Epoch 6/10
500/500 [==============================] - 44s 87ms/step - loss: 2.3471 - val_loss: 2.4384
Epoch 7/10
500/500 [==============================] - 44s 87ms/step - loss: 2.2646 - val_loss: 2.3949
Epoch 8/10
500/500 [==============================] - 44s 87ms/step - loss: 2.1875 - val_loss: 2.3542
Epoch 9/10
500/500 [==============================] - 44s 87ms/step - loss: 2.1130 - val_loss: 2.3220
Epoch 10/10
500/500 [==============================] - 44s 87ms/step - loss: 2.041

In [ ]:
print(generate_text(model1, tokenizer, init_sentence="<start> he"))

print(generate_text(model2, tokenizer, init_sentence="<start> he"))

print(generate_text(model3, tokenizer, init_sentence="<start> he"))

print(generate_text(model4, tokenizer, init_sentence="<start> he"))

print(generate_text(model5, tokenizer, init_sentence="<start> he"))

print(generate_text(model1, tokenizer, init_sentence="<start> some"))

print(generate_text(model2, tokenizer, init_sentence="<start> some"))

print(generate_text(model3, tokenizer, init_sentence="<start> some"))

print(generate_text(model4, tokenizer, init_sentence="<start> some"))

print(generate_text(model5, tokenizer, init_sentence="<start> some"))

print(generate_text(model1, tokenizer, init_sentence="<start> i love"))

print(generate_text(model2, tokenizer, init_sentence="<start> i love"))

print(generate_text(model3, tokenizer, init_sentence="<start> i love"))

print(generate_text(model4, tokenizer, init_sentence="<start> i love"))

print(generate_text(model5, tokenizer, init_sentence="<start> i love"))

print(generate_text(model1, tokenizer, init_sentence="<start> abc"))

print(generate_text(model2, tokenizer, init_sentence="<start> aiffel"))

print(generate_text(model3, tokenizer, init_sentence="<start> green"))

print(generate_text(model4, tokenizer, init_sentence="<start> grass"))

print(generate_text(model5, tokenizer, init_sentence="<start> star"))


<start> he s got a <unk> <end> 
<start> he s a <unk> <end> 
<start> he s a monster <end> 
<start> he s just a girl , i m a <unk> <end> 
<start> he s got a <unk> <end> 
<start> some people want diamond rings <end> 
<start> some <unk> <unk> <end> 
<start> some people want diamond rings <end> 
<start> some people want diamond rings <end> 
<start> some people want diamond rings <end> 
<start> i love you , i m not gonna crack <end> 
<start> i love you , i m a <unk> <end> 
<start> i love you , i love you <end> 
<start> i love the way you lie <end> 
<start> i love you so much , i m not gonna be alright <end> 
<start> <unk> <unk> <end> 
<start> <unk> <unk> <unk> <end> 
<start> green eggs and ham . <end> 
<start> grass s <unk> <end> 
<start> star wars , nigga , i m a <unk> <end> 


In [ ]:
print(generate_text(model4, tokenizer, init_sentence="<start> ask"))

<start> ask me what you wanna do <end> 


In [ ]:
print(generate_text(model1, tokenizer, init_sentence="<start> "))

<start> i m the one gon hold you down <end> 


In [ ]:
print(generate_text(model4, tokenizer, init_sentence="<start> star"))

<start> star wars , <unk> <unk> <end> 


In [ ]:
print(generate_text(model5, tokenizer, init_sentence="<start> star"))

<start> star wars , nigga , i m a <unk> <end> 


# 입력 데이터에 문장 길이를 길게 준 결과 조금 만족스러운 문장이 나왔다.

## 그리고 알게 된점은 무조건 길게 한다고 더 좋은 결과가 나오지 않는다는 점을 알게 되었다.

# 결론 1: 데이터 전처리는 컴퓨터가 쉽게 계산할 수 있도록 하자              
# 결론 2: 자연어 처리는 공부가 더 필요할건 같다.